In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('./ml-25m/genome-scores.csv')

In [3]:
sorted_relevance = df[df['relevance'] > 0.5].sort_values(by=['movieId', 'relevance'], ascending=[True, False])

In [4]:
movie_id_names = pd.read_csv('./ml-25m/movies.csv')

In [11]:
movie_tags = pd.merge(sorted_relevance, movie_id_names, on='movieId')

In [12]:
tags = pd.read_csv('./ml-25m/genome-tags.csv')

In [13]:
links = pd.read_csv('./ml-25m/links.csv')

In [14]:
movie_tags = pd.merge(movie_tags, tags, on='tagId')

In [15]:
movie_tags = pd.merge(movie_tags, links, on='movieId')

In [16]:
movie_tags = movie_tags[['imdbId', 'title', 'genres', 'tag']]

In [17]:
movie_tags['imdbId'] = movie_tags['imdbId'].apply(lambda x: 'tt' + str(x).zfill(7))

In [18]:
import sqlite3
conn = sqlite3.connect('../fastapi/query/movie.db')
cursor = conn.cursor()

In [19]:
movie_id_in_db = pd.read_sql_query('SELECT MovieID FROM Movie', conn)

In [20]:
movie_tags = movie_tags[movie_tags['imdbId'].isin(movie_id_in_db['MovieID'])]

In [21]:
movie_tags_grouped = movie_tags.groupby('tag').agg({
    'title': lambda x: ' | '.join(x), 
}).reset_index()

In [22]:
movie_tags_grouped.to_csv('./tag_movies.csv', index=False)

In [34]:
import pandas as pd
tag_prompt = pd.read_csv('./pg/tag_results.csv')

In [35]:
tag_prompt = pd.merge(tag_prompt, movie_tags_grouped, on='tag')

In [36]:
tag_prompt['prompt'] = tag_prompt['prompt'].apply(lambda x: "'"+ x + "'")

In [37]:
tag_prompt['title'] = tag_prompt['title'].apply(lambda x: "<<"+ x + ">>")

In [38]:
tag_prompt.to_csv('./pg/tag_prompt_titles.csv', index=False)

In [45]:
import re
def check_format(text):
    pattern = r'^Welcome to MovieSearch[\s\S]*?\n- [^\n]+'
    return bool(re.match(pattern, text))

In [46]:
check_format("""Welcome to MovieSearch. 

The James Bond film series is an iconic and long-running franchise, with the first film, Dr. No (1962), kicking off the phenomenon. The series follows the adventures of the British secret agent, 007, and has spanned decades with multiple actors portraying the titular character. Some of the most popular and well-known Bond films include:

Goldfinger (1964) Skyfall (2012)
- Casino Royale (2006)
- GoldenEye (1995)
- The Spy Who Loved Me (1977)

The most recent addition to the series is No Time to Die (2021), which serves as a fitting conclusion to Daniel Craig's tenure as James Bond. 

Would you like me to suggest more spy-thriller movies similar to the 007 series?""")

False